In [ ]:
import datetime
import pandas as pd
import ipywidgets as widgets

In [ ]:
arbo_v = widgets.Combobox(
    placeholder = 'Selecione uma: ',
    options = ["Dengue", "Chikungunya", "Zika"],
    description = 'Arbovirose: ',
    ensure_option = True
)

display(arbo_v)

In [ ]:
arbo_v.value = str.lower(arbo_v.value)

In [ ]:
aux = datetime.date.today()
aux = int(aux.strftime("%Y"))

if arbo_v.value == 'dengue':
    j = 10
else:
    j = 5

anos_aux = []
for i in range (aux-j, aux):
    anos_aux.append(str(i))

In [ ]:
ano = widgets.Combobox(
    placeholder = 'Selecione o ano: ',
    options = anos_aux,
    description = 'Ano: ',
    ensure_option = True
)

display(ano)

In [ ]:
anos_aux.remove(ano.value)

In [ ]:
ano_comparado = widgets.Combobox(
    placeholder = 'Selecione o ano: ',
    options = anos_aux,
    description = 'Ano: ',
    ensure_option = True
)

display(ano_comparado)

In [ ]:
scr = f"../scr/{arbo_v.value}/"

if arbo_v.value == 'dengue':
    arq = scr + "casos_municipio_10.csv"
else:
    arq = scr + "casos_municipio_5.csv"

df = pd.read_csv(arq, encoding = "utf-8")
df = df.drop(columns=['Unnamed: 0'])

In [ ]:
censo = pd.read_csv("../scr/BA - IBGE 2010.csv")
df["População 2010"] = 0

for i in range (len(df)):
    for j in range (len(censo)):
        if df.loc[i, "Município"] == censo.loc[j, "Município"]:
            df.loc[i, "População 2010"] = censo.loc[j, "População 2010"]

df["Casos por 100.000 habitantes"] = round(df.loc[:, f'{ano.value}']/df.loc[:, "População 2010"] * 100000, 1)

In [ ]:
if arbo_v.value == 'dengue':
    max_anos = 13
else:
    max_anos = 8

total_casos = []
data = []
for elem in (list(df.columns)[2:max_anos]):
    total_casos.append(df.loc[:, elem].sum())
    data.append(elem)

total_casos = pd.DataFrame(total_casos, data)

In [ ]:
if arbo_v.value == "dengue":
    df["Obitos"] = 0
    obito = pd.read_csv("../scr/dengue/obitos_municipio_10.csv")
    for i in range (len(df)):
        for j in range (len(obito)):
            if df.loc[i, "Município"] == obito.loc[j, "Município"]:
                df.loc[i, "Obitos"] = obito.loc[j, ano.value]
    
    df["Mortalidade por 100.000 habitantes"] = round(df.loc[:, "Obitos"]/df.loc[:, "População 2010"] * 100000, 1)

In [ ]:
df[f'Taxa de crescimento dos anos {ano.value} e {ano_comparado.value}'] = round((df.loc[:, ano.value] - df.loc[:, ano_comparado.value]) / df.loc[:, ano_comparado.value] * 100, 1)

In [ ]:
if arbo_v.value == "dengue":
    aux = df.nlargest(10, 'Casos por 100.000 habitantes')[['Codigo', 'Município', 'População 2010', f'{ano.value}', 'Obitos', 'Casos por 100.000 habitantes', 'Mortalidade por 100.000 habitantes', f'Taxa de crescimento dos anos {ano.value} e {ano_comparado.value}']]
else:
    aux = df.nlargest(10, 'Casos por 100.000 habitantes')[['Codigo', 'Município', 'População 2010', f'{ano.value}', 'Casos por 100.000 habitantes', f'Taxa de crescimento dos anos {ano.value} e {ano_comparado.value}']]

In [ ]:
str('A média e desvio padrão de casos de {0} dos anos {1} até {2} foi de {3} e {4}').format(arbo_v.value, total_casos.iloc[0, :].name, total_casos.iloc[-1, :].name, round(total_casos.iloc[:, 0].mean(), 1), round(total_casos.iloc[:, 0].std(), 1))

In [ ]:
taxa_20_19 = round((total_casos.loc["2020"][0] - total_casos.loc["2019"][0]) / total_casos.loc["2019"][0] * 100, 1)
taxa_20_21 = round((total_casos.loc["2020"][0] - total_casos.loc["2021"][0]) / total_casos.loc["2021"][0] * 100, 1)

str('Taxa de crescimento de {0} dos anos de 2019 e 2021 comparados com 2020 foi de {1}% e {2}%').format(arbo_v.value, taxa_20_19, taxa_20_21)

In [ ]:
total_casos["População 2010"] = df["População 2010"].sum()
total_casos["Casos por 100.000 habitantes"] = round(total_casos.iloc[:, 0]/total_casos.loc[:, "População 2010"] * 100000, 1)

In [ ]:
if arbo_v.value == 'dengue':
    obitos = pd.read_csv("../scr/dengue/obitos_municipio_10.csv")
    total_casos["Obitos"] = list(obitos.iloc[:, 3:14].sum())

In [ ]:
total_casos = total_casos.rename(columns={0: "Casos"})

In [ ]:
total_casos

In [ ]:
aux